In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
from statsmodels.graphics.mosaicplot import mosaic
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt, numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import AgglomerativeClustering
from matplotlib.colors import ListedColormap
from sklearn import metrics
import warnings
import sys
import random
if not sys.warnoptions:
    warnings.simplefilter("ignore")
np.random.seed(42)

In [3]:
data_events = pd.read_csv("data_events_sample.csv")
purchase_events = pd.read_csv("purchase_events_sample.csv")


In [4]:
data_events.head()

,TIMESTAMP_EVENT,PRICE,PROPOSITION,USER_CLIENT_NUMBER,ARTICLE_CATEGORIE,USER_SESSION_ID,PROMOTION_LABEL,PAGE_NAME,PAGE_SECTION,PAGE_SECTION_POSITION,...,PRODUCT_TYPE,DEVICE_INFO_BRAND,DEVICE_INFO_TYPE,DEVICE_INFO_BROWSER,USER_SALES_GROUP,USER_SEGMENT,USER_SALES_DISTRICT,USER_PROMOTIONS_ALLOWED,EVENT,PROMOTION_PRICE
0,2022-03-24T15:17:27.041Z,39.1248,67795,763956,"1,101,781,000",47920,20%25%20korting,Shopping Cart,Recommended Products,2.0,...,Meepakker,(Windows) PC,COMPUTER,Chrome 99.0.4844.74,Hotel Restaurant,restauratief,Groningen,1,VIEW,31.2972
1,2022-01-25T12:31:18.664Z,62.3832,192360,591921,"1,101,791,010",70923,no promotion,others,not recommended,0.0,...,others,,COMPUTER,Chrome 97.0.4692.77,Zorg Overig Algemeen,zorg,Hoofddorp,1,ADD_TO_CARD,0.0000
2,2022-03-16T14:23:28.490Z,16.7376,284988,144567,"1,091,621,316",256691,no promotion,others,not recommended,0.0,...,others,(Windows) PC,COMPUTER,Chrome 99.0.4844.51,Rest. dag/avondkaart,restauratief,Hoofddorp,1,ADD_TO_CARD,0.0000
3,2022-01-23T15:38:28.392Z,11.0484,248504,43389,"1,061,271,285",318858,20%25%20korting,Shopping Cart,Recommended Products,1.0,...,Meepakker,,COMPUTER,Chrome 94.0.4606.124,Verpl/Verz Restaur,zorg,Schiedam,1,VIEW,8.8440
4,2022-03-13T15:01:33.264Z,42.5172,40050,185712,"1,101,761,030",167729,no promotion,others,not recommended,0.0,...,others,Apple Macintosh,COMPUTER,Chrome 89.0.4389.90,Restaur. dagkaart,restauratief,Tilburg,1,ADD_TO_CARD,0.0000


In [5]:
purchase_events.head()

,PROPOSITION,USER_CLIENT_NUMBER,DATE,AMOUNT
0,692611,280187,2021-10-19 00:00:00,55.22
1,706870,615463,2021-11-09 00:00:00,33.86
2,607268,98679,2022-04-06 00:00:00,75.40
3,211805,803697,2022-03-24 00:00:00,188.18
4,453929,379568,2022-02-09 00:00:00,148.50


In [6]:
data_events['TIMESTAMP_EVENT'] = pd.to_datetime(data_events['TIMESTAMP_EVENT'])
purchase_events['DATE'] = pd.to_datetime(purchase_events['DATE'])

# Merge the datasets on common keys 'PROPOSITION' and 'USER_CLIENT_NUMBER'
full_data = pd.merge(data_events, purchase_events, on=['PROPOSITION', 'USER_CLIENT_NUMBER'], how='left')


In [7]:
full_data

,TIMESTAMP_EVENT,PRICE,PROPOSITION,USER_CLIENT_NUMBER,ARTICLE_CATEGORIE,USER_SESSION_ID,PROMOTION_LABEL,PAGE_NAME,PAGE_SECTION,PAGE_SECTION_POSITION,...,DEVICE_INFO_TYPE,DEVICE_INFO_BROWSER,USER_SALES_GROUP,USER_SEGMENT,USER_SALES_DISTRICT,USER_PROMOTIONS_ALLOWED,EVENT,PROMOTION_PRICE,DATE,AMOUNT
0,2022-03-24 15:17:27.041000+00:00,39.1248,67795,763956,"1,101,781,000",47920,20%25%20korting,Shopping Cart,Recommended Products,2.0,...,COMPUTER,Chrome 99.0.4844.74,Hotel Restaurant,restauratief,Groningen,1,VIEW,31.2972,NaT,NaN
1,2022-01-25 12:31:18.664000+00:00,62.3832,192360,591921,"1,101,791,010",70923,no promotion,others,not recommended,0.0,...,COMPUTER,Chrome 97.0.4692.77,Zorg Overig Algemeen,zorg,Hoofddorp,1,ADD_TO_CARD,0.0000,2022-04-07,162.89
2,2022-03-16 14:23:28.490000+00:00,16.7376,284988,144567,"1,091,621,316",256691,no promotion,others,not recommended,0.0,...,COMPUTER,Chrome 99.0.4844.51,Rest. dag/avondkaart,restauratief,Hoofddorp,1,ADD_TO_CARD,0.0000,NaT,NaN
3,2022-01-23 15:38:28.392000+00:00,11.0484,248504,43389,"1,061,271,285",318858,20%25%20korting,Shopping Cart,Recommended Products,1.0,...,COMPUTER,Chrome 94.0.4606.124,Verpl/Verz Restaur,zorg,Schiedam,1,VIEW,8.8440,NaT,NaN
4,2022-03-13 15:01:33.264000+00:00,42.5172,40050,185712,"1,101,761,030",167729,no promotion,others,not recommended,0.0,...,COMPUTER,Chrome 89.0.4389.90,Restaur. dagkaart,restauratief,Tilburg,1,ADD_TO_CARD,0.0000,2022-03-21,96.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1841977,2022-02-28 09:19:58.145000+00:00,20.4204,118713,293304,"1,091,611,119",343485,no promotion,others,not recommended,0.0,...,PHONE,Stock Browser,Cafetaria +,fastservice,Helmond,1,ADD_TO_CARD,0.0000,2021-10-05,302.15
1841978,2022-02-28 09:19:58.145000+00:00,20.4204,118713,293304,"1,091,611,119",343485,no promotion,others,not recommended,0.0,...,PHONE,Stock Browser,Cafetaria +,fastservice,Helmond,1,ADD_TO_CARD,0.0000,2021-11-26,161.15
1841979,2022-03-14 22:21:01.536000+00:00,10.0056,897236,928012,"1,081,411,508",97075,no promotion,Purchase List,Recommended Products,1.0,...,COMPUTER,Safari 15.1,Aziatisch Cafetaria,fastservice,Groningen,1,VIEW,0.0000,NaT,NaN
1841980,2022-01-25 19:41:41.332000+00:00,71.9268,182389,16692,"1,091,661,550",161405,40%25%20korting,Shopping Cart,Recommended Products,4.0,...,COMPUTER,Safari 15.1,Restaur. dagkaart,restauratief,Emmen,1,VIEW,43.1508,NaT,NaN


 Feature Engineering
segmentation:

Recency (R): Time since last purchase.
Frequency (F): Total number of purchases.
Monetary (M): Total spending.

In [8]:
# Group by user and aggregate data
user_data = full_data.groupby('USER_CLIENT_NUMBER').agg({
    'DATE': 'max',  # Most recent purchase date
    'PROPOSITION': 'count',  # Frequency of purchases
    'AMOUNT': 'sum'  # Total amount spent
}).rename(columns={'DATE': 'Recent_Purchase_Date', 'PROPOSITION': 'Total_Purchases', 'AMOUNT': 'Total_Spent'})

# Calculate recency in days from the most recent date in the dataset
most_recent_date = full_data['DATE'].max()
user_data['Recency'] = (most_recent_date - user_data['Recent_Purchase_Date']).dt.days


In [9]:
# Customer Segmentation: quantile-based segmentation method 

# Use quantiles to segment each RFM feature
quantiles = user_data[['Recency', 'Total_Purchases', 'Total_Spent']].quantile(q=[0.25, 0.5, 0.75])
def rfm_segment(x, p, d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]:
        return 3
    else:
        return 4

user_data['R_Score'] = user_data['Recency'].apply(rfm_segment, args=('Recency', quantiles))
user_data['F_Score'] = user_data['Total_Purchases'].apply(rfm_segment, args=('Total_Purchases', quantiles))
user_data['M_Score'] = user_data['Total_Spent'].apply(rfm_segment, args=('Total_Spent', quantiles))


With segmented data, analyze patterns and develop targeted marketing strategies:

High Value Customers: Top RFM scores. Develop loyalty programs or premium offers.
At-Risk Customers: High monetary and frequency scores but low recency score. Re-engage with personalized promotions.
Low-Value Customers: Low scores across RFM. Explore up-sell and cross-sell opportunities to increase their value.

In [10]:
# Define a simple function to assign recommendations based on RFM scores
def assign_recommendations(row):
    if row['R_Score'] == 4 and row['F_Score'] == 4 and row['M_Score'] == 4:
        return 'Premium Product Line'
    elif row['R_Score'] == 1 and row['F_Score'] >= 3:
        return 'Re-engagement Campaign'
    else:
        return 'Regular Promotions'

user_data['Recommendations'] = user_data.apply(assign_recommendations, axis=1)


In [11]:
user_data
# Use the insights from the RFM segmentation to fuel your recommendation engine, which will suggest products based on the user's segment.

,Recent_Purchase_Date,Total_Purchases,Total_Spent,Recency,R_Score,F_Score,M_Score,Recommendations
USER_CLIENT_NUMBER,,,,,,,,
193,NaT,1,0.00,NaN,4,1,1,Regular Promotions
230,2022-04-28,141,605.72,0.0,1,3,2,Re-engagement Campaign
272,NaT,9,0.00,NaN,4,1,1,Regular Promotions
436,2022-04-14,72,587.02,14.0,3,3,2,Regular Promotions
467,NaT,3,0.00,NaN,4,1,1,Regular Promotions
...,...,...,...,...,...,...,...,...
999388,2022-04-22,21,178.28,6.0,3,2,2,Regular Promotions
999500,2022-04-26,46,198.12,2.0,1,2,2,Regular Promotions
999513,NaT,6,0.00,NaN,4,1,1,Regular Promotions


In [15]:
def can_fit_model(y_data):
    """Check if there are at least two classes in y_data."""
    return len(set(y_data)) > 1


if len(X_train[chosen_arm]) > 1 and can_fit_model(y_train[chosen_arm]):
    models[chosen_arm].fit(np.array(X_train[chosen_arm]), np.array(y_train[chosen_arm]))
    fit_models[chosen_arm] = True


In [16]:
def simulate_user_response(action):
    # Adjust the probability to ensure variability in outcomes
    probabilities = [0.7, 0.3] if action == 0 else [0.3, 0.7]
    return np.random.choice([0, 1], p=probabilities)


# Initializing training data with examples from both classes
for arm in range(n_arms):
    X_train[arm] = [np.random.normal(size=n_features), np.random.normal(size=n_features)]
    y_train[arm] = [0, 1]  # Ensuring both classes are present
    models[arm].fit(np.array(X_train[arm]), np.array(y_train[arm]))
    fit_models[arm] = True
